In [6]:
import cv2
import os
from tool.darknet2pytorch import Darknet
from models import Yolov4
from demo import *
from tool.utils import *
import torch
from datetime import datetime

In [3]:
cfgfile = 'cfg/yolov4.cfg'
weightfile = 'weights/yolov4.weights'
imgfile = 'data/dog.jpg'
vidfile = '../vid_short.mp4'

cap = cv2.VideoCapture(vidfile)
fps = int(cap.get(cv2.CAP_PROP_FPS))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(height, width)
print(fps)


720 1280
24


In [4]:
# code heavily based on https://github.com/Tianxiaomo/pytorch-YOLOv4/blob/4ccef0ec8fe984e059378813e33b3740929e0c19/tool/utils.py#L99
def our_plot_boxes_cv2(img, boxes, savename=None, class_names=None, color=None, allowed_classes=None):
    img = np.copy(img)
    colors = np.array([[1, 0, 1], [0, 0, 1], [0, 1, 1], [0, 1, 0], [1, 1, 0], [1, 0, 0]], dtype=np.float32)

    def get_color(c, x, max_val):
        ratio = float(x) / max_val * 5
        i = int(math.floor(ratio))
        j = int(math.ceil(ratio))
        ratio = ratio - i
        r = (1 - ratio) * colors[i][c] + ratio * colors[j][c]
        return int(r * 255)

    width = img.shape[1]
    height = img.shape[0]
    for i in range(len(boxes)):
        box = boxes[i]
        x1 = int(box[0] * width)
        y1 = int(box[1] * height)
        x2 = int(box[2] * width)
        y2 = int(box[3] * height)

        if color:
            rgb = color
        else:
            rgb = (255, 0, 0)
        if len(box) >= 7 and class_names:
            cls_conf = box[5]
            cls_id = box[6]
            
            # skip box if it is not for a class in allowed_classes
            if allowed_classes and class_names[cls_id] not in allowed_classes:
                continue
            
#             print('%s: %f' % (class_names[cls_id], cls_conf))
            classes = len(class_names)
            offset = cls_id * 123457 % classes
            red = get_color(2, offset, classes)
            green = get_color(1, offset, classes)
            blue = get_color(0, offset, classes)
            if color is None:
                rgb = (red, green, blue)
            img = cv2.putText(img, class_names[cls_id], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1.2, rgb, 1)
        img = cv2.rectangle(img, (x1, y1), (x2, y2), rgb, 1)
    if savename:
#         print("save plot results to %s" % savename)
        cv2.imwrite(savename, img)
    return img

# code heavily based on https://github.com/Tianxiaomo/pytorch-YOLOv4/blob/4ccef0ec8fe984e059378813e33b3740929e0c19/demo.py#L25
def our_detect(cfgfile, weightfile, vidfile, use_cuda=True):
    m = Darknet(cfgfile)
    m.load_weights(weightfile)
    print('Loading weights from %s... Done!' % (weightfile))

    if use_cuda:
        m.cuda()

    num_classes = m.num_classes
    if num_classes == 20:
        namesfile = 'data/voc.names'
    elif num_classes == 80:
        namesfile = 'data/coco.names'
    else:
        namesfile = 'data/x.names'
    class_names = load_class_names(namesfile)
    cap = cv2.VideoCapture(vidfile)
    video = cv2.VideoWriter('inference.avi',cv2.VideoWriter_fourcc(*'DIVX'),24,(width,height))
    i_frame = 0
    while cap.isOpened():
        i_frame += 1
        ret, frame = cap.read()
        if not ret:
            break
        sized = cv2.resize(frame, (m.width, m.height))
        sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)

        for i in range(2):
            start = time.time()
            boxes = do_detect(m, sized, 0.4, 0.6, use_cuda)
            finish = time.time()
            if i == 1:
                print('Frame %s: Predicted in %f seconds.' % (i_frame, (finish - start)))

        os.makedirs('predictions', exist_ok=True)
#         plot_boxes_cv2(frame, boxes[0], savename='predictions/frame%i.jpg' % i_frame, class_names=class_names)
        frame_with_box = our_plot_boxes_cv2(frame, boxes[0], class_names=class_names, allowed_classes=['person'])
        video.write(frame_with_box)
        
#         if i_frame == 50:
#             break

    cv2.destroyAllWindows()
    video.release()

In [13]:
our_detect(cfgfile, weightfile, vidfile)

convalution havn't activate linear
convalution havn't activate linear
convalution havn't activate linear
Loading weights from weights/yolov4.weights... Done!
-----------------------------------
           Preprocess : 0.005004
      Model Inference : 0.227207
-----------------------------------
-----------------------------------
       max and argmax : 0.007006
                  nms : 0.001001
Post processing total : 0.008007
-----------------------------------
-----------------------------------
           Preprocess : 0.004003
      Model Inference : 0.086079
-----------------------------------
-----------------------------------
       max and argmax : 0.007007
                  nms : 0.002002
Post processing total : 0.009009
-----------------------------------
Frame 1: Predicted in 0.104094 seconds.
-----------------------------------
           Preprocess : 0.004004
      Model Inference : 0.087078
-----------------------------------
-----------------------------------
       max

-----------------------------------
           Preprocess : 0.003005
      Model Inference : 0.087076
-----------------------------------
-----------------------------------
       max and argmax : 0.006005
                  nms : 0.002002
Post processing total : 0.008007
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.087079
-----------------------------------
-----------------------------------
       max and argmax : 0.006005
                  nms : 0.002002
Post processing total : 0.008007
-----------------------------------
Frame 14: Predicted in 0.101092 seconds.
-----------------------------------
           Preprocess : 0.003002
      Model Inference : 0.088081
-----------------------------------
-----------------------------------
       max and argmax : 0.007006
                  nms : 0.001001
Post processing total : 0.008007
-----------------------------------
--------------------------------

-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.088080
-----------------------------------
-----------------------------------
       max and argmax : 0.006006
                  nms : 0.002002
Post processing total : 0.008008
-----------------------------------
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.086078
-----------------------------------
-----------------------------------
       max and argmax : 0.006006
                  nms : 0.002002
Post processing total : 0.008008
-----------------------------------
Frame 27: Predicted in 0.100092 seconds.
-----------------------------------
           Preprocess : 0.004004
      Model Inference : 0.085077
-----------------------------------
-----------------------------------
       max and argmax : 0.007006
                  nms : 0.001001
Post processing total : 0.008007
-----------------------------------
--------------------------------

-----------------------------------
       max and argmax : 0.008007
                  nms : 0.002002
Post processing total : 0.010009
-----------------------------------
-----------------------------------
           Preprocess : 0.008007
      Model Inference : 0.087079
-----------------------------------
-----------------------------------
       max and argmax : 0.008008
                  nms : 0.002002
Post processing total : 0.010009
-----------------------------------
Frame 40: Predicted in 0.110100 seconds.
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.085078
-----------------------------------
-----------------------------------
       max and argmax : 0.008007
                  nms : 0.002002
Post processing total : 0.010009
-----------------------------------
-----------------------------------
           Preprocess : 0.003004
      Model Inference : 0.085076
-----------------------------------
--------------------------------

-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.086078
-----------------------------------
-----------------------------------
       max and argmax : 0.007006
                  nms : 0.002002
Post processing total : 0.009008
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.089082
-----------------------------------
-----------------------------------
       max and argmax : 0.006005
                  nms : 0.002002
Post processing total : 0.008007
-----------------------------------
Frame 53: Predicted in 0.103093 seconds.
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.087080
-----------------------------------
-----------------------------------
       max and argmax : 0.010009
                  nms : 0.002002
Post processing total : 0.012011
-----------------------------------
--------------------------------

...

-----------------------------------
       max and argmax : 0.007006
                  nms : 0.001001
Post processing total : 0.008007
-----------------------------------
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.086078
-----------------------------------
-----------------------------------
       max and argmax : 0.007006
                  nms : 0.001001
Post processing total : 0.008007
-----------------------------------
Frame 417: Predicted in 0.101092 seconds.
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.086078
-----------------------------------
-----------------------------------
       max and argmax : 0.007006
                  nms : 0.001001
Post processing total : 0.008007
-----------------------------------
-----------------------------------
           Preprocess : 0.009008
      Model Inference : 0.084076
-----------------------------------
-------------------------------

-----------------------------------
       max and argmax : 0.008007
                  nms : 0.001001
Post processing total : 0.009008
-----------------------------------
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.085077
-----------------------------------
-----------------------------------
       max and argmax : 0.007016
                  nms : 0.001001
Post processing total : 0.008017
-----------------------------------
Frame 430: Predicted in 0.100101 seconds.
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.086078
-----------------------------------
-----------------------------------
       max and argmax : 0.008017
                  nms : 0.001993
Post processing total : 0.010010
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.086078
-----------------------------------
-------------------------------

-----------------------------------
           Preprocess : 0.002993
      Model Inference : 0.087079
-----------------------------------
-----------------------------------
       max and argmax : 0.007006
                  nms : 0.002002
Post processing total : 0.009008
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.086078
-----------------------------------
-----------------------------------
       max and argmax : 0.007006
                  nms : 0.001001
Post processing total : 0.008008
-----------------------------------
Frame 443: Predicted in 0.101092 seconds.
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.087079
-----------------------------------
-----------------------------------
       max and argmax : 0.009008
                  nms : 0.001001
Post processing total : 0.010010
-----------------------------------
-------------------------------

-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.087079
-----------------------------------
-----------------------------------
       max and argmax : 0.006005
                  nms : 0.002002
Post processing total : 0.008007
-----------------------------------
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.085077
-----------------------------------
-----------------------------------
       max and argmax : 0.006006
                  nms : 0.001001
Post processing total : 0.007006
-----------------------------------
Frame 456: Predicted in 0.099090 seconds.
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.085078
-----------------------------------
-----------------------------------
       max and argmax : 0.009008
                  nms : 0.002002
Post processing total : 0.011010
-----------------------------------
-------------------------------

-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.088081
-----------------------------------
-----------------------------------
       max and argmax : 0.008007
                  nms : 0.001001
Post processing total : 0.009008
-----------------------------------
-----------------------------------
           Preprocess : 0.003002
      Model Inference : 0.085078
-----------------------------------
-----------------------------------
       max and argmax : 0.007006
                  nms : 0.001001
Post processing total : 0.008007
-----------------------------------
Frame 469: Predicted in 0.100091 seconds.
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.086079
-----------------------------------
-----------------------------------
       max and argmax : 0.007006
                  nms : 0.001001
Post processing total : 0.008007
-----------------------------------
-------------------------------